In [145]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
df = pd.read_csv('cancer_mama.csv')


In [146]:
#Classes  ['recorrencia'],['idade'],['menopausa'],['tumor'],['nodulos'],['capsula'],['maligno'],['seio'],['quadrante_seio'],['radioterapia']
df

,recorrencia,idade,menopausa,tumor,nodulos,capsula,maligno,seio,quadrante_seio,radioterapia
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...,...
281,recurrence-events,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,recurrence-events,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,recurrence-events,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,recurrence-events,40-49,ge40,30-34,3-5,no,3,left,left_low,no


In [147]:
#Retirando uma coluna que não tem importancia para o resultado
# Como mostrado nesse artigo https://www.cancer.org/cancer/breast-cancer/about/what-is-breast-cancer.html
df = df.drop('seio', axis=1)
# ja o quadrante é mais importante, podendo definir o tipo de cirurgia realizada como diz nesse artigo https://www.breastcancer.org/symptoms/types/quadrant.

In [148]:
# Função para encontrar a porcentagem dos dados faltantes,
def percentual(dados, coluna):
  percentuais = dados.loc[pd.isna(dados[coluna]),coluna].shape[0]
  total = dados[coluna].shape[0]

  return (percentuais/total)*100

códigos individuais para testar a porcentagem de cada um

In [149]:
percentual(df,'idade')

0.0

In [150]:
percentual(df,'menopausa')

0.0

In [151]:
percentual(df,'maligno')

0.0

In [152]:
percentual(df,'tumor')

0.0

In [153]:
percentual(df,'recorrencia')

0.0

In [154]:
percentual(df,'radioterapia')

0.0

In [155]:
percentual(df,'capsula')

0.0

In [156]:
percentual(df,'quadrante_seio')

0.0

In [157]:
percentual(df,'nodulos')

0.0

In [158]:
# Processo para remover valores encontrados que estão faltando
faltantes = []
for coluna in df:
    todos_faltantes = len(df.loc[df[coluna]=='?'])
    faltantes.append(todos_faltantes)

In [159]:
df.loc[df['capsula']=='?', 'capsula'] = np.nan
df.loc[df['quadrante_seio']=='?', 'quadrante_seio'] = np.nan

In [160]:
df = df[~pd.isna(df.quadrante_seio)].reset_index()
df = df.drop(columns='index')

Mostrando os uniques de todas as colunas

In [161]:
df['idade'].unique()

array(['30-39', '40-49', '60-69', '50-59', '70-79', '20-29'], dtype=object)

In [162]:
df['menopausa'].unique()

array(['premeno', 'ge40', 'lt40'], dtype=object)

In [163]:
df['tumor'].unique()

array(['30-34', '20-24', '15-19', '0-4', '25-29', '50-54', '10-14',
       '40-44', '35-39', '5-9', '45-49'], dtype=object)

In [164]:
df['nodulos'].unique()

array(['0-2', '6-8', '9-11', '3-5', '15-17', '12-14', '24-26'],
      dtype=object)

In [165]:
df['capsula'].unique()

array(['no', 'yes', nan], dtype=object)

In [166]:
df['maligno'].unique()

array([3, 2, 1])

In [167]:
df['quadrante_seio'].unique()

array(['left_low', 'right_up', 'left_up', 'right_low', 'central'],
      dtype=object)

In [168]:
df['radioterapia'].unique()

array(['no', 'yes'], dtype=object)

In [169]:
#convertendo os campos categóricos em on hot

def one_hot(categorias, valor):
    categorias = list(set(categorias))
    index = categorias.index(valor)
    vetor = np.zeros(len(categorias))
    vetor[index] = 1
    return vetor

In [170]:
df.recorrencia = [one_hot(df.recorrencia, _) for _ in df.recorrencia]
df.idade = [one_hot(df.idade, _) for _ in df.idade]
df.menopausa = [one_hot(df.menopausa, _) for _ in df.menopausa]
df.maligno = [one_hot(df.maligno, _) for _ in df.maligno]
df.capsula = [one_hot(df.capsula, _) for _ in df.capsula]
df.quadrante_seio = [one_hot(df.quadrante_seio, _) for _ in df.quadrante_seio]
df.tumor = [one_hot(df.tumor, _) for _ in df.tumor]
df.nodulos = [one_hot(df.nodulos, _) for _ in df.nodulos]
df.radioterapia = [one_hot(df.radioterapia, _) for _ in df.radioterapia]

In [171]:
#apagando a coluna das respostas

resultado = df.recorrencia
df = df.drop(columns = 'recorrencia')

In [172]:
#concatenando as colunas para conseguir utilizar os dados
def vetorizacao(idade,menopausa,tumor,nodulos,capsula,maligno,quadrante_seio,radioterapia):
    return  idade.tolist() + menopausa.tolist() + tumor.tolist() + nodulos.tolist() \
+ capsula.tolist() + maligno.tolist() + quadrante_seio.tolist() + radioterapia.tolist()

In [173]:
entrada = [vetorizacao(*_) for _ in df.values]

separando os valores e os resultados em dois arquvios diferentes para comparalos no futuro, determinando a precisão

In [174]:
np.savetxt("dados_processados.csv", entrada, delimiter=',')

In [175]:
np.savetxt("resultado.csv", resultado.tolist(), delimiter=',')

In [176]:
x = np.loadtxt("dados_processados.csv", delimiter=',')
y = np.loadtxt("resultado.csv", delimiter=',')

In [177]:
tamanho_treino = int(x.shape[0]*0.75)

In [178]:
x_treino = x[ :tamanho_treino]
y_treino = y[ :tamanho_treino]

x_s = x[tamanho_treino: ]
y_s = y[tamanho_treino: ]

In [183]:
redeneural = MLPClassifier(verbose=True,
                           max_iter=10000,
                           tol=0.000001,
                           activation='relu',
                           #hiden_layer_sizes=(100,100),
                           learning_rate_init=0.0001)

In [184]:
redeneural.fit(x,y)

A saída de streaming foi truncada nas últimas 5000 linhas.
Iteration 1975, loss = 0.46951913
Iteration 1976, loss = 0.46930929
Iteration 1977, loss = 0.46910146
Iteration 1978, loss = 0.46888109
Iteration 1979, loss = 0.46867533
Iteration 1980, loss = 0.46846531
Iteration 1981, loss = 0.46826435
Iteration 1982, loss = 0.46810135
Iteration 1983, loss = 0.46789563
Iteration 1984, loss = 0.46770611
Iteration 1985, loss = 0.46747649
Iteration 1986, loss = 0.46726399
Iteration 1987, loss = 0.46703629
Iteration 1988, loss = 0.46680865
Iteration 1989, loss = 0.46666973
Iteration 1990, loss = 0.46639456
Iteration 1991, loss = 0.46618939
Iteration 1992, loss = 0.46594405
Iteration 1993, loss = 0.46574265
Iteration 1994, loss = 0.46553112
Iteration 1995, loss = 0.46533833
Iteration 1996, loss = 0.46512141
Iteration 1997, loss = 0.46490479
Iteration 1998, loss = 0.46469794
Iteration 1999, loss = 0.46449078
Iteration 2000, loss = 0.46429603
Iteration 2001, loss = 0.46407172
Iteration 2002, loss = 

MLPClassifier(learning_rate_init=0.0001, max_iter=10000, tol=1e-06,
              verbose=True)

In [185]:
y_pred = redeneural.predict(x_treino)

In [186]:
precisao = accuracy_score(y_treino, y_pred)
print("Precisão: {:.2f}%".format(precisao * 100))

Precisão: 96.24%
